# Creating the London and Merseyside CSVs

Load in libraries:

In [5]:
import os
import pandas as pd 
import geopandas as gpd
# from shapely.geometry import Point


### Load the data
2022 Police Data

https://data.police.uk/data/

In [6]:
path = '../../Data/London_Merseyside_Police.UK_Data'    

# find all csv file paths
csv_names = []

# loop all csv folders
for folder in os.listdir(path):
        
            # extract the CSVs in each folder
            folder_contents = os.listdir(f'{path}/{folder}')
            
            # loop through each file
            for file in folder_contents:
                
                # add each csv file name    
                csv_names.append(f'{path}/{folder}/{file}')

# print the number of CSVs -- 108
print(f'There are {len(csv_names)} different CSVs')

There are 72 different CSVs


### Extract the CSV's

In [7]:
crimes, outcomes, stops = [], [], []

# loop through the csvs
for csv in csv_names:
    
    # collect the stop and search data
    if "stop-and-search.csv" in csv:
        stops.append(pd.read_csv(csv))

    # collect the crime data
    elif "street.csv" in csv:
        crimes.append(pd.read_csv(csv))

    # collect the outcomes of the crime data
    elif "outcomes.csv" in csv:
        outcomes.append(pd.read_csv(csv))

    # else ERROR
    else:
        print('error with CSV')

# concat each df
# outcomes_df = pd.concat(outcomes, ignore_index=True)
crimes_df = pd.concat(crimes, ignore_index=True)
stops_df = pd.concat(stops, ignore_index=True)


### Combine the crime and outcomes by 'CRIME ID'

In [8]:
# merge on crime id
# crimes_df = crimes_df.merge(outcomes_df[['Crime ID', 'Outcome type']], 
#                             on='Crime ID', 
#                             how='left')
# # look at the data
# display(crimes_df)

### Convert to datetime


In [9]:
# convert to datetime 
# crimes_df['Date'] = pd.to_datetime(crimes_df['Month'], format= '%Y-%m')

# # convert to datetime and remove timezone
# stops_df['Date'] = pd.to_datetime(stops_df['Date']).dt.tz_convert(None)

# display(stops_df)

## Map LSOA to coords

https://borders.ukdataservice.ac.uk/

In [10]:

# Update this path to the actual location of the shapefile
# path_to_shapefile = '../../Data/LSOA_data/BoundaryData/england_lsoa_2021.shp'
path_to_shapefile = '../../Data/LSOA_Data/LSOA_boundaries/LSOA_2021_EW_BSC_V4.shp'

lsoa_gdf = gpd.read_file(path_to_shapefile)

# convert to Long, Lat from ordinate survey corrds
lsoa_gdf_4326 = lsoa_gdf.to_crs("EPSG:4326")

lsoa_gdf_4326 = lsoa_gdf_4326[['LSOA21CD', 'geometry']]

display(lsoa_gdf_4326)

,LSOA21CD,geometry
0,E01000001,"POLYGON ((-0.09477 51.52059, -0.09548 51.51544..."
1,E01000002,"POLYGON ((-0.08813 51.51941, -0.09548 51.51544..."
2,E01000003,"POLYGON ((-0.09456 51.52205, -0.09276 51.52139..."
3,E01000005,"POLYGON ((-0.07592 51.5159, -0.07397 51.51445,..."
4,E01000006,"POLYGON ((0.09326 51.53787, 0.0936 51.53767, 0..."
...,...,...
35667,W01002036,"POLYGON ((-3.1771 51.44702, -3.17618 51.44261,..."
35668,W01002037,"POLYGON ((-3.16646 51.44662, -3.17179 51.44302..."
35669,W01002038,"POLYGON ((-3.3434 51.3898, -3.34925 51.38719, ..."
35670,W01002039,"POLYGON ((-3.33577 51.38279, -3.33867 51.38189..."


In [11]:

# Assume df is your DataFrame with long/lat columns
gdf_points = gpd.GeoDataFrame(
    stops_df,
    geometry=gpd.points_from_xy(stops_df['Longitude'], stops_df['Latitude']),
    crs='EPSG:4326'
)


joined = gpd.sjoin(gdf_points, 
                   lsoa_gdf_4326, 
                   how='left', 
                   predicate='intersects')


display(joined)

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing,geometry,index_right,LSOA21CD
0,Person search,2022-01-01T00:04:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,False,POINT EMPTY,NaN,NaN
1,Person search,2022-01-01T00:07:45+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,False,POINT EMPTY,NaN,NaN
2,Person search,2022-01-01T00:12:22+00:00,NaN,NaN,53.406618,-2.983878,Male,10-17,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,False,POINT (-2.98388 53.40662),31803.0,E01033760
3,Person search,2022-01-01T00:17:54+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,False,POINT EMPTY,NaN,NaN
4,Person search,2022-01-01T00:22:40+00:00,NaN,NaN,53.403914,-2.981499,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Arrest,False,True,POINT (-2.9815 53.40391),31800.0,E01033756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233586,Person and Vehicle search,2022-12-17T21:35:00+00:00,0.0,NaN,51.631560,-0.195095,Male,25-34,Asian/Asian British - Any other Asian background,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,NaN,POINT (-0.1951 51.63156),265.0,E01000279
233587,Person search,2022-12-17T21:57:00+00:00,0.0,NaN,NaN,NaN,Male,25-34,Other ethnic group - Not stated,White,Police and Criminal Evidence Act 1984 (section 1),Stolen goods,Community resolution,NaN,NaN,POINT EMPTY,NaN,NaN
233588,Person search,2022-12-17T22:00:00+00:00,0.0,NaN,51.514365,-0.143408,Female,over 34,White - Any other White background,White,Police and Criminal Evidence Act 1984 (section 1),Stolen goods,A no further action disposal,NaN,NaN,POINT (-0.14341 51.51436),33709.0,E01035716
233589,Person search,2022-12-17T22:16:00+00:00,0.0,NaN,NaN,NaN,Male,10-17,Black/African/Caribbean/Black British - Caribbean,Black,Police and Criminal Evidence Act 1984 (section 1),Stolen goods,A no further action disposal,NaN,NaN,POINT EMPTY,NaN,NaN


# Group LSOA Statistics

In [12]:

dataset = joined.groupby('LSOA21CD').agg(
   
    Stop_Count=('Date', 'count'),
    Stop_count_drugs=('Object of search', lambda x: (x.str.contains("drugs", case=False, na=False).sum())), 
    # Arrest_outcome=('Outcome', lambda x: (x.str.contains('Arrest', case=False, na=False).sum())),
    geometry=('geometry', 'first')  # Keep the geometry as the first point in each group (change to 'mean' or other if necessary)


).round(2).reset_index()



dataset


,LSOA21CD,Stop_Count,Stop_count_drugs,geometry
0,E01000001,10,2,POINT (-0.09591 51.52035)
1,E01000002,2,1,POINT (-0.09455 51.52066)
2,E01000005,35,25,POINT (-0.07306 51.51118)
3,E01000006,39,34,POINT (0.08705 51.5383)
4,E01000007,189,156,POINT (0.07407 51.53963)
...,...,...,...,...
5935,W01000414,1,1,POINT (-2.96827 53.04106)
5936,W01000423,5,5,POINT (-2.96585 53.03762)
5937,W01000425,2,2,POINT (-2.98013 53.04583)
5938,W01001931,3,3,POINT (-2.9887 53.04611)


## Clean dual jurisdiction cases 
### Add missing LSOAs with no Stop and Search DATA

https://geoportal.statistics.gov.uk/datasets/ons::lsoa-2021-to-local-authority-districts-april-2023-best-fit-lookup-in-ew/explore


In [13]:
LSOA_names = pd.read_csv('../../Data/LSOA_data/ONS/2021LSOA_list.csv')

# Define a list of Merseyside boroughs 
#https://www.ons.gov.uk/visualisations/areas/E11000002/ 

# Define a list of London boroughs 
# https://www.ons.gov.uk/visualisations/areas/E12000007/


boroughs = [
    "Barking and Dagenham", "Barnet", "Bexley", "Brent", "Bromley", "Camden",
    "Croydon", "Ealing", "Enfield", "Greenwich", "Hackney", "Hammersmith and Fulham",
    "Haringey", "Harrow", "Havering", "Hillingdon", "Hounslow", "Islington",
    "Kensington and Chelsea", "Kingston upon Thames", "Lambeth", "Lewisham",
    "Merton", "Newham", "Redbridge", "Richmond upon Thames", "Southwark",
    "Sutton", "Tower Hamlets", "Waltham Forest", "Wandsworth", "Westminster",
    "City of London" , 
    'Liverpool', 'Wirral', 'Sefton', 'Knowsley', 'St. Helens'
]

LSOAs = LSOA_names[LSOA_names['LAD23NM'].isin(boroughs)]
LSOAs = LSOAs[['LSOA21CD', 'LAD23NM']]
LSOAs.columns = ['LSOA code', 'Borough']
LSOAs

dataset = dataset.merge(LSOAs, 
                        right_on='LSOA code',
                        left_on='LSOA21CD', 
                        how='right')
dataset

,LSOA21CD,Stop_Count,Stop_count_drugs,geometry,LSOA code,Borough
0,E01006434,127.0,111.0,POINT (-2.86793 53.47524),E01006434,Knowsley
1,E01006435,98.0,83.0,POINT (-2.87845 53.47415),E01006435,Knowsley
2,E01006436,373.0,326.0,POINT (-2.87491 53.47936),E01006436,Knowsley
3,E01006437,534.0,434.0,POINT (-2.88482 53.48091),E01006437,Knowsley
4,E01006438,38.0,32.0,POINT (-2.8547 53.46165),E01006438,Knowsley
...,...,...,...,...,...,...
5912,E01004660,4.0,0.0,POINT (-0.15905 51.52289),E01004660,Westminster
5913,E01004661,278.0,119.0,POINT (-0.16786 51.51923),E01004661,Westminster
5914,E01004662,23.0,9.0,POINT (-0.16236 51.52232),E01004662,Westminster
5915,E01004663,108.0,49.0,POINT (-0.16411 51.51742),E01004663,Westminster


## LSOA % EthnicMinority
https://www.nomisweb.co.uk/sources/census_2021_bulk

In [14]:
LSOA_ethnic = pd.read_csv('../../Data/LSOA_data/ONS/2021census_ethnic.csv') 

nonWhite =  (1- LSOA_ethnic['Ethnic group: White'] / LSOA_ethnic.iloc[:,3])*100
LSOA_ethnic
LSOA_ethnic = LSOA_ethnic.iloc[:, [2,3]]
LSOA_ethnic['nonWhite_pop'] = round(nonWhite,2)
LSOA_ethnic.columns = ['LSOA code', 'population', 'EthnicMinority']
LSOA_ethnic

dataset = dataset.merge(LSOA_ethnic, on='LSOA code', how='left')
dataset

,LSOA21CD,Stop_Count,Stop_count_drugs,geometry,LSOA code,Borough,population,EthnicMinority
0,E01006434,127.0,111.0,POINT (-2.86793 53.47524),E01006434,Knowsley,1518,1.91
1,E01006435,98.0,83.0,POINT (-2.87845 53.47415),E01006435,Knowsley,1524,3.35
2,E01006436,373.0,326.0,POINT (-2.87491 53.47936),E01006436,Knowsley,1457,3.29
3,E01006437,534.0,434.0,POINT (-2.88482 53.48091),E01006437,Knowsley,1387,3.03
4,E01006438,38.0,32.0,POINT (-2.8547 53.46165),E01006438,Knowsley,1153,5.29
...,...,...,...,...,...,...,...,...
5912,E01004660,4.0,0.0,POINT (-0.15905 51.52289),E01004660,Westminster,1430,41.54
5913,E01004661,278.0,119.0,POINT (-0.16786 51.51923),E01004661,Westminster,1998,49.80
5914,E01004662,23.0,9.0,POINT (-0.16236 51.52232),E01004662,Westminster,1318,31.64
5915,E01004663,108.0,49.0,POINT (-0.16411 51.51742),E01004663,Westminster,1417,37.47


In [15]:
dataset.drop(columns={'geometry',
                      'LSOA21CD'},
            inplace=True)
dataset

,Stop_Count,Stop_count_drugs,LSOA code,Borough,population,EthnicMinority
0,127.0,111.0,E01006434,Knowsley,1518,1.91
1,98.0,83.0,E01006435,Knowsley,1524,3.35
2,373.0,326.0,E01006436,Knowsley,1457,3.29
3,534.0,434.0,E01006437,Knowsley,1387,3.03
4,38.0,32.0,E01006438,Knowsley,1153,5.29
...,...,...,...,...,...,...
5912,4.0,0.0,E01004660,Westminster,1430,41.54
5913,278.0,119.0,E01004661,Westminster,1998,49.80
5914,23.0,9.0,E01004662,Westminster,1318,31.64
5915,108.0,49.0,E01004663,Westminster,1417,37.47


## Income Domain Rank

https://www.gov.uk/government/statistics/indices-of-deprivation-2019-income-and-employment-domains-combined-for-england-and-wales

In [16]:
# read in the data (first sheet)
LSOA_IDR = pd.read_excel('../../Data/LSOA_data/GOV/Income_domain_scores.ods', 
              sheet_name=1, 
              engine="odf",
              usecols=['LSOA Code (2011)', 
                       'Income Domain Score'])

# only use 
LSOA_IDR = LSOA_IDR[['LSOA Code (2011)', 'Income Domain Score']]
LSOA_IDR.columns = ['LSOA code', 'Income Domain Score']

LSOA_IDR

,LSOA code,Income Domain Score
0,E01000001,0.007
1,E01000002,0.034
2,E01000003,0.086
3,E01000005,0.211
4,E01000006,0.117
...,...,...
34748,W01001954,0.049
34749,W01001955,0.420
34750,W01001956,0.038
34751,W01001957,0.234


In [17]:
dataset = dataset.merge(LSOA_IDR, on='LSOA code', how='left')
dataset

,Stop_Count,Stop_count_drugs,LSOA code,Borough,population,EthnicMinority,Income Domain Score
0,127.0,111.0,E01006434,Knowsley,1518,1.91,0.361
1,98.0,83.0,E01006435,Knowsley,1524,3.35,0.370
2,373.0,326.0,E01006436,Knowsley,1457,3.29,0.384
3,534.0,434.0,E01006437,Knowsley,1387,3.03,0.326
4,38.0,32.0,E01006438,Knowsley,1153,5.29,0.099
...,...,...,...,...,...,...,...
5912,4.0,0.0,E01004660,Westminster,1430,41.54,0.035
5913,278.0,119.0,E01004661,Westminster,1998,49.80,0.126
5914,23.0,9.0,E01004662,Westminster,1318,31.64,0.043
5915,108.0,49.0,E01004663,Westminster,1417,37.47,0.092


## Crime Data

In [18]:
# Calculate the total crime count and drug crime count by LSOA code
crime_counts = crimes_df.groupby('LSOA code').size().reset_index(name='crime_sum')
drug_counts = crimes_df[crimes_df['Crime type'] == 'Drugs'].groupby('LSOA code').size().reset_index(name='drug_crime_sum')

# Merge both crime counts into a single dataset
dataset = dataset.merge(crime_counts, on='LSOA code', how='left')
dataset = dataset.merge(drug_counts, on='LSOA code', how='left')

# Display the final dataset
dataset


,Stop_Count,Stop_count_drugs,LSOA code,Borough,population,EthnicMinority,Income Domain Score,crime_sum,drug_crime_sum
0,127.0,111.0,E01006434,Knowsley,1518,1.91,0.361,228.0,23.0
1,98.0,83.0,E01006435,Knowsley,1524,3.35,0.370,149.0,18.0
2,373.0,326.0,E01006436,Knowsley,1457,3.29,0.384,307.0,58.0
3,534.0,434.0,E01006437,Knowsley,1387,3.03,0.326,921.0,72.0
4,38.0,32.0,E01006438,Knowsley,1153,5.29,0.099,91.0,7.0
...,...,...,...,...,...,...,...,...,...
5912,4.0,0.0,E01004660,Westminster,1430,41.54,0.035,344.0,1.0
5913,278.0,119.0,E01004661,Westminster,1998,49.80,0.126,737.0,17.0
5914,23.0,9.0,E01004662,Westminster,1318,31.64,0.043,306.0,2.0
5915,108.0,49.0,E01004663,Westminster,1417,37.47,0.092,333.0,15.0



## House Prices
https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/meanpricepaidbylowerlayersuperoutputareahpssadataset47



In [19]:

# Read the 5th sheet (index 4)
xls = pd.read_excel('../../Data/LSOA_data/ONS/2022mean_house_price.xls', 
                    sheet_name=5, 
                    engine='xlrd')  # or use 'openpyxl' if the file is .xlsx

# Set the 5th row as column headers
headings = xls.iloc[4]

# Slice the data after the 5th row (exclude the header row)
house_prices = xls.iloc[5:]

# Assign the correct column names
house_prices.columns = headings

# Select the relevant columns
house_prices = house_prices[['Local authority code', 'LSOA code', 
                             'Year ending Mar 2022', 'Year ending Jun 2022', 
                             'Year ending Sep 2022', 'Year ending Dec 2022']]

# Calculate the mean house price (ignoring non-numeric values)
house_prices['mean_house_price'] = (
    house_prices.iloc[:, 2:]  # Selecting only the columns with prices
    .apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN
    .mean(axis=1)  # Calculate the mean across the rows
    .round()  # Round to nearest integer
)

# Keep only the necessary columns (LSOA code and mean house price)
house_prices = house_prices[['LSOA code', 'mean_house_price']]

# Merge with the existing dataset
dataset = dataset.merge(house_prices, on='LSOA code',  how='left')

# Display the final merged dataset
dataset


,Stop_Count,Stop_count_drugs,LSOA code,Borough,population,EthnicMinority,Income Domain Score,crime_sum,drug_crime_sum,mean_house_price
0,127.0,111.0,E01006434,Knowsley,1518,1.91,0.361,228.0,23.0,123955.0
1,98.0,83.0,E01006435,Knowsley,1524,3.35,0.370,149.0,18.0,134664.0
2,373.0,326.0,E01006436,Knowsley,1457,3.29,0.384,307.0,58.0,111733.0
3,534.0,434.0,E01006437,Knowsley,1387,3.03,0.326,921.0,72.0,119648.0
4,38.0,32.0,E01006438,Knowsley,1153,5.29,0.099,91.0,7.0,331221.0
...,...,...,...,...,...,...,...,...,...,...
5912,4.0,0.0,E01004660,Westminster,1430,41.54,0.035,344.0,1.0,1655186.0
5913,278.0,119.0,E01004661,Westminster,1998,49.80,0.126,737.0,17.0,1283005.0
5914,23.0,9.0,E01004662,Westminster,1318,31.64,0.043,306.0,2.0,1427386.0
5915,108.0,49.0,E01004663,Westminster,1417,37.47,0.092,333.0,15.0,1817948.0


# Split Data into Cities

In [20]:
# list of Merseyside boroughs
m_boroughs = ['Liverpool', 'Wirral', 'Sefton', 'Knowsley', 'St. Helens']

# Create a dummy column based on the 'borough' column
dataset['Police_Dept'] = dataset['Borough'].apply(lambda x: 'Merseyside' if x in m_boroughs else 'London')

# Display the updated DataFrame
display(dataset)


,Stop_Count,Stop_count_drugs,LSOA code,Borough,population,EthnicMinority,Income Domain Score,crime_sum,drug_crime_sum,mean_house_price,Police_Dept
0,127.0,111.0,E01006434,Knowsley,1518,1.91,0.361,228.0,23.0,123955.0,Merseyside
1,98.0,83.0,E01006435,Knowsley,1524,3.35,0.370,149.0,18.0,134664.0,Merseyside
2,373.0,326.0,E01006436,Knowsley,1457,3.29,0.384,307.0,58.0,111733.0,Merseyside
3,534.0,434.0,E01006437,Knowsley,1387,3.03,0.326,921.0,72.0,119648.0,Merseyside
4,38.0,32.0,E01006438,Knowsley,1153,5.29,0.099,91.0,7.0,331221.0,Merseyside
...,...,...,...,...,...,...,...,...,...,...,...
5912,4.0,0.0,E01004660,Westminster,1430,41.54,0.035,344.0,1.0,1655186.0,London
5913,278.0,119.0,E01004661,Westminster,1998,49.80,0.126,737.0,17.0,1283005.0,London
5914,23.0,9.0,E01004662,Westminster,1318,31.64,0.043,306.0,2.0,1427386.0,London
5915,108.0,49.0,E01004663,Westminster,1417,37.47,0.092,333.0,15.0,1817948.0,London


### Rename columns for ease

In [21]:
dataset.columns

Index(['Stop_Count', 'Stop_count_drugs', 'LSOA code', 'Borough', 'population',
       'EthnicMinority', 'Income Domain Score', 'crime_sum', 'drug_crime_sum',
       'mean_house_price', 'Police_Dept'],
      dtype='object')

In [22]:
cols = ['StopCount', 'StopCountDrugs', 'LSOA21CD', 'Borough', 'PopulationLSOA', 
        'EthnicMinority', 'IncomeDomainScore', 'CrimeSum', 'DrugCrimeSum', 
        'MeanHousePrice', 'PoliceDept']

dataset.columns = cols

display(dataset)

,StopCount,StopCountDrugs,LSOA21CD,Borough,PopulationLSOA,EthnicMinority,IncomeDomainScore,CrimeSum,DrugCrimeSum,MeanHousePrice,PoliceDept
0,127.0,111.0,E01006434,Knowsley,1518,1.91,0.361,228.0,23.0,123955.0,Merseyside
1,98.0,83.0,E01006435,Knowsley,1524,3.35,0.370,149.0,18.0,134664.0,Merseyside
2,373.0,326.0,E01006436,Knowsley,1457,3.29,0.384,307.0,58.0,111733.0,Merseyside
3,534.0,434.0,E01006437,Knowsley,1387,3.03,0.326,921.0,72.0,119648.0,Merseyside
4,38.0,32.0,E01006438,Knowsley,1153,5.29,0.099,91.0,7.0,331221.0,Merseyside
...,...,...,...,...,...,...,...,...,...,...,...
5912,4.0,0.0,E01004660,Westminster,1430,41.54,0.035,344.0,1.0,1655186.0,London
5913,278.0,119.0,E01004661,Westminster,1998,49.80,0.126,737.0,17.0,1283005.0,London
5914,23.0,9.0,E01004662,Westminster,1318,31.64,0.043,306.0,2.0,1427386.0,London
5915,108.0,49.0,E01004663,Westminster,1417,37.47,0.092,333.0,15.0,1817948.0,London


### Save to CSV

In [23]:
dataset.to_csv('../../Data/Data Sets/raw_dataset.csv', index=False)
